In [ ]:
from keras.models import load_model
import cv2
import tensorflow as tf
import os
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
from PIL import Image
from tensorflow.keras.preprocessing import image

# Initialize Flask application
app = Flask(__name__)
CORS(app, origins='http://127.0.0.1:5500')

# Load models
model = tf.keras.models.load_model(os.path.join('models','model_hybrid_model_mark04.h5'))

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/rate', methods=['POST'])
def detect():
    # Get image from request
    img_file = request.files['image']
    if img_file and allowed_file(img_file.filename):
        img = Image.open(img_file.stream)
        if img.format == 'PNG':
            img = img.convert('RGB')
        img = image.img_to_array(img)

        resize = tf.image.resize(img, (90,120))
        result_array = model.predict(np.expand_dims(resize/255, 0))
        result_value = result_array[0][0] * 100

        print("Prediction: {:.2f}%".format(result_value))
        response = {"prediction": "{:.2f}%".format(result_value)}
        
        return jsonify(response)
    else:
        return jsonify({"error": "Invalid file type"}), 400

if __name__ == '__main__':
    app.run(port=5062)
